1. Introduction to CustomOp


Need to create subclasses of `CustomOp` to provide execution, code generation and other functionality in FINN.

In [5]:
from finn.custom_op import CustomOp
dir(CustomOp)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 'execute_node',
 'get_nodeattr',
 'get_nodeattr_types',
 'infer_node_datatype',
 'make_shape_compatible_op',
 'set_nodeattr',
 'verify_node']

Some points of importance:

1. `CustomOp` instances (in Python) are not meant to store any data, only provide functionality on top of data stored in ONNX. Each `CustomOp` instance has a member `self.onnx_node` which gives access to the ONNX `NodeProto` with attributes. There is also a custom attribute setter/getter system in `CustomOp` to make this process easier.

2. `CustomOp` subclasses need to implement the methods above (those not starting with underscore) -- see custom_op/__init__.py for full details.

3. `CustomOp` subclasses must be registered with the custom operator registry in `custom_op/registry.py` to be usable with the FINN framework.

## A Simple CustomOp Example

Let's make a simple CustomOp that raises its input to a given exponent (specified as attribute). For now it'll only work in Python, but later we'll add C++ execution capability too.

In [25]:
from onnx import helper
import numpy as np

class MyPythonPowerOp(CustomOp):
    
    # here we use the CustomOp attribute system to make it easier
    # to set/get custom attributes on this node
    def get_nodeattr_types(self):
        return {
            # each entry is:
            # name of attribute : (dtype, required, default value)
            # dtype follows the ONNX attribute protobuf so
            # "i" is int, "s" is string, "f" is float,
            # "ints" is a list of integers...
            # also good practice to document what each attribute does here:
            
            # which integer power to raise the input to
            "exponent" : ("i", True, 0),
            # execution mode : currently only python
            "exec_mode" : ("s", True, "python"),
        }
    
    # return an ONNX node that has the same shape inference behavior
    # here we want in shape = out shape, so we use the ONNX ReLU
    # node to mimic its shape inference behavior
    # we have access to the entire ModelWrapper to help make this decision
    # (the parameter called model)
    def make_shape_compatible_op(self, model):
        node = self.onnx_node
        # make a Relu node connected to the same in-out tensors to get
        # shape inference
        # a general-purpose alternative is to use a Constant node that 
        # produces the desired shape
        return helper.make_node("Relu", [node.input[0]], [node.output[0]])

    # used for FINN DataType inference: set the output tensors' datatypes
    # accordingly for this node
    # here we assume input datatype = output datatype
    # we have access to the entire ModelWrapper to help make this decision
    # (the parameter called model)
    def infer_node_datatype(self, model):
        node = self.onnx_node
        # data type stays the same
        dtype = model.get_tensor_datatype(node.input[0])
        model.set_tensor_datatype(node.output[0], dtype)
    
    # execute this node
    # context: used for both input and output, dictionary of named
    #          tensors
    # graph: the ONNX GraphProto (ModelWrapper.graph), generally 
    #         not needed to execute a single node
    def execute_node(self, context, graph):
        exec_mode = self.get_nodeattr("exec_mode")
        if exec_mode == "python":
            # get names of node input and output tensors
            i_name = self.onnx_node.input[0]
            o_name = self.onnx_node.output[0]
            # grab input tensor from context
            i_tensor = context[i_name]
            # get which power to raise to from attribute
            expnt = self.get_nodeattr("exponent")
            # compute and put output into context
            o_tensor = np.power(i_tensor, expnt)
            context[o_name] = o_tensor
        else:
            raise Exception("Only python exec_mode is supported")
        
    # can use to do a sanity check of all the node's properties
    # optional, not implemented here
    def verify_node(self):
        pass
        
        

Recall that to make it this usable in FINN we also have to register in in the custom op registry:

In [26]:
from finn.custom_op.registry import custom_op

custom_op["MyPythonPowerOp"] = MyPythonPowerOp

## Let's Try Out our CustomOp

We'll manually build a small ONNX graph containing our node in order to try out some of the functionality. This would normally go into the unit test for this CustomOp.

In [27]:
from finn.core.modelwrapper import ModelWrapper
from onnx import TensorProto

def make_graph(ishape, exp):
    inp = helper.make_tensor_value_info(
        "inp", TensorProto.FLOAT, ishape
    )
    outp = helper.make_tensor_value_info(
        "outp", TensorProto.FLOAT, ishape
    )

    custom_node = helper.make_node(
        # op type string in ONNX, what we used to register the custom op
        "MyPythonPowerOp",
        # name of input tensor
        ["inp"],
        # name of output tensor
        ["outp"],
        # needed for custom ops
        domain="finn",
        # set up attributes
        exponent = int(exp),
        exec_mode = "python"
    )

    graph = helper.make_graph(
        nodes=[custom_node], name="custom_graph", inputs=[inp], outputs=[outp]
    )
    model = helper.make_model(graph, producer_name="custom-model")
    return ModelWrapper(model)

In [32]:
# generate a small graph with our custom op
input_shape = (1, 2, 4)
ret_model = make_graph(input_shape, 2)
ret_model.model.graph.node

[input: "inp"
output: "outp"
op_type: "MyPythonPowerOp"
attribute {
  name: "exec_mode"
  s: "python"
  type: STRING
}
attribute {
  name: "exponent"
  i: 2
  type: INT
}
domain: "finn"
]

In [29]:
from finn.core.datatype import DataType
from finn.util.basic import gen_finn_dt_tensor

# generate a random input of e.g signed 4-bit values
random_input = gen_finn_dt_tensor(DataType.INT4, input_shape)
random_input


array([[[-5.,  1., -2.,  7.],
        [ 3., -1.,  4.,  5.]]], dtype=float32)

In [31]:
from finn.core.onnx_exec import execute_onnx

# run with FINN's execute_onnx
inp_dict = {"inp" : random_input}
ret = execute_onnx(ret_model, inp_dict)
ret

{'outp': array([[[25.,  1.,  4., 49.],
         [ 9.,  1., 16., 25.]]], dtype=float32)}

## A CustomOp with C++ Generation

In [ ]:
from finn.util.basic import make_build_dir, CppBuilder

# derive from our previous example
class MyMixedPowerOp(MyPythonPowerOp):
    
    # here we use the CustomOp attribute system to make it easier
    # to set/get custom attributes on this node
    def get_nodeattr_types(self):
        return {
            # each entry is:
            # name of attribute : (dtype, required, default value)
            # dtype follows the ONNX attribute protobuf so
            # "i" is int, "s" is string, "f" is float,
            # "ints" is a list of integers...
            # also good practice to document what each attribute does here:
            
            # which integer power to raise the input to
            "exponent" : ("i", True, 0),
            # execution mode : python or c++
            "exec_mode" : ("s", True, "python"),
            # code generation directory
            "codegen_dir" : ("s", False, ""),
        }
    
    def my_custom_cpp_gen(self):
        build_dir = make_build_dir(prefix="my_custom_op")
        # set attribute for codegen dir
        self.set_nodeattr("codegen_dir", build_dir)
        # generate some C++ code
        cpp_code = """
#include <iostream>
#include <fstream>
using namespace std;
#define EXPONENT %d

int main(int argc, char **argv) {
    ifstream infile("input.txt");
    ofstream outfile("output.txt");
    
    float elem;
    while (infile >> elem)
    {
        float res = 1.0;
        for(int i=0; i < EXPONENT; i++) {
            res *= elem;
        }
        outfile << res << "\n";
    }

    return 0;
}
        """ % (self.get_nodeattr("exponent"))
        with open(build_dir+"/top.cpp", "w") as f:
            f.write(cpp_code)
        builder = CppBuilder()
        # to enable additional debug features please uncommand the next line
        builder.append_includes("--std=c++11")
        builder.append_includes("-O3")
        builder.append_sources(build_dir + "/*.cpp")
        builder.set_executable_path(build_dir + "/node_model")
        builder.build(code_gen_dir)
    
    # execute this node
    # context: used for both input and output, dictionary of named
    #          tensors
    # graph: the ONNX GraphProto (ModelWrapper.graph), generally 
    #         not needed to execute a single node
    def execute_node(self, context, graph):
        exec_mode = self.get_nodeattr("exec_mode")
        # get names of node input and output tensors
        i_name = self.onnx_node.input[0]
        o_name = self.onnx_node.output[0]
        # grab input tensor from context
        i_tensor = context[i_name]
        # get which power to raise to from attribute
        expnt = self.get_nodeattr("exponent")
        if exec_mode == "python":
            # compute and put output into context
            o_tensor = np.power(i_tensor, expnt)
            context[o_name] = o_tensor
        elif exec_mode == "c++":
            build_dir = self.get_nodeattr("codegen_dir")
            # save input as txt, could preprocess, change layout etc..
            np.savetxt(build_dir+"/input.txt")
            bash_command = ["bash", build_dir+"/node_model"]
            proc_run = subprocess.Popen(bash_command, stdout=subprocess.PIPE)
            proc_run.communicate()
            np.loadtxt(build_dir+"/output.txt")
            
        else:
            raise Exception("Only python exec_mode is supported")
        
    # can use to do a sanity check of all the node's properties
    # optional, not implemented here
    def verify_node(self):
        pass
        
        

## Implement a code generation transformation


In [ ]:
from finn.transformation import Transformation
